In [ ]:
# deeplabv3_eval_multi_dirs.py
import os, glob, cv2, numpy as np, json
from tqdm import tqdm
import torch
from PIL import Image
from mmseg.apis import init_model, inference_model
from mmseg.utils import register_all_modules

# ===================== User settings =====================
CONFIG_FILE = r'C:/Users/heheh/mmsegmentation/configs/deeplabv3/deeplabv3_r50-d8_4xb4-80k_coco-stuff164k-512x512.py'
CHECKPOINT  = r'C:/Users/heheh/mmsegmentation/checkpoints/deeplabv3_r50-d8_512x512_4x4_80k_coco-stuff164k_20210709_163016-88675c24.pth'

# Name -> image folder (jpg). Use matching COCO val2017 basenames.
IMAGE_DIRS = {
    'Clean'       : r'C:/Users/heheh/val2017/val2017',
    'Semantic_Adv': r'C:/Users/heheh/mmsegmentation/data/semantic_adv_deeplabv3', 
    'instance_pgd': r'C:/Users/heheh/mmdetection/data/coco/instance_maskrcnn_adv',
    'Panoptic_Adv': r'C:/Users/heheh/mmdetection/data/coco/panoptic_fpn_adv',
}
# Ground-truth PNG label maps (COCO-Stuff 164k)
GT_DIR = r'C:/Users/heheh/mmsegmentation/data/coco_stuff164k/annotations/val2017'

# Evaluate only a subset for speed (None = all in the intersection)
MAX_SAMPLES = 5000

COCO_STUFF_LABEL_MAP = {
    0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:8, 9:9, 10:10,
    11:11, 13:12, 14:13, 15:14, 16:15, 17:16, 18:17, 19:18, 20:19, 21:20,
    22:21, 23:22, 24:23, 25:24, 27:25, 28:26, 31:27, 32:28, 33:29, 34:30,
    35:31, 36:32, 37:33, 38:34, 39:35, 40:36, 41:37, 42:38, 43:39, 44:40,
    46:41, 47:42, 48:43, 49:44, 50:45, 51:46, 52:47, 53:48, 54:49, 55:50,
    56:51, 57:52, 58:53, 59:54, 60:55, 61:56, 62:57, 63:58, 64:59, 65:60,
    67:61, 70:62, 72:63, 73:64, 74:65, 75:66, 76:67, 77:68, 78:69, 79:70,
    80:71, 81:72, 82:73, 84:74, 85:75, 86:76, 87:77, 88:78, 89:79, 90:80,
    92:81, 93:82, 94:83, 95:84, 96:85, 97:86, 98:87, 99:88, 100:89, 101:90,
    102:91, 103:92, 104:93, 105:94, 106:95, 107:96, 108:97, 109:98, 110:99, 111:100,
    112:101, 113:102, 114:103, 115:104, 116:105, 117:106, 118:107, 119:108, 120:109, 121:110,
    122:111, 123:112, 124:113, 125:114, 126:115, 127:116, 128:117, 129:118, 130:119, 131:120,
    132:121, 133:122, 134:123, 135:124, 136:125, 137:126, 138:127, 139:128, 140:129, 141:130,
    142:131, 143:132, 144:133, 145:134, 146:135, 147:136, 148:137, 149:138, 150:139, 151:140,
    152:141, 153:142, 154:143, 155:144, 156:145, 157:146, 158:147, 159:148, 160:149, 161:150,
    162:151, 163:152, 164:153, 165:154, 166:155, 167:156, 168:157, 169:158, 170:159, 171:160,
    172:161, 173:162, 174:163, 175:164, 176:165, 177:166, 178:167, 179:168, 180:169, 181:170,
    182:171, 255:255
}

# Save CSVs
SAVE_DIR = r'C:/Users/heheh/mmsegmentation/data/eval_out'
SAVE_SUMMARY_CSV = True
SAVE_PER_CLASS_IOU_CSV = True

IGNORE_INDEX = 255
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# =========================================================
def build_label_lut(mapping: dict, ignore_index=255):
    size = max(max(mapping.keys()), ignore_index) + 1   # 256 works, but compute robustly
    lut = np.full(size, ignore_index, dtype=np.int64)
    for k, v in mapping.items():
        lut[k] = v
    return lut

# replace your OpenCV-based GT loader with this
def load_gt_png(path, remap_lut=None, num_classes=None, ignore_index=255):
    with Image.open(path) as img:
        # preserve palette indices; don't convert to RGB
        if img.mode in ('P', 'L', 'I'):
            gt = np.array(img)
        else:
            gt = np.array(img.convert('P'))
    if remap_lut is not None:
        # apply LUT safely
        max_idx = remap_lut.shape[0] - 1
        gt = remap_lut[np.clip(gt, 0, max_idx)]
    # clamp any stray ids to ignore
    if num_classes is not None:
        bad = (gt != ignore_index) & ((gt < 0) | (gt >= num_classes))
        if bad.any():
            gt = np.where(bad, ignore_index, gt)
    return gt.astype(np.int64)

def basenames_in(dir_path, ext):
    paths = glob.glob(os.path.join(dir_path, f'*.{ext}'))
    return set(os.path.splitext(os.path.basename(p))[0] for p in paths)

def fast_confusion_update(conf_mat, pred, gt, num_classes, ignore_index=255):
    mask = gt != ignore_index
    if not np.any(mask):
        return
    gt_v = gt[mask]
    pr_v = pred[mask]
    valid = (gt_v >= 0) & (gt_v < num_classes) & (pr_v >= 0) & (pr_v < num_classes)
    if not np.any(valid):
        return
    gt_v = gt_v[valid]
    pr_v = pr_v[valid]
    idx = gt_v * num_classes + pr_v
    counts = np.bincount(idx, minlength=num_classes * num_classes)
    conf_mat += counts.reshape(num_classes, num_classes)

def metrics_from_confusion(conf_mat):
    inter = np.diag(conf_mat)
    area_gt = conf_mat.sum(1)
    area_pr = conf_mat.sum(0)
    union = area_gt + area_pr - inter

    iou = np.divide(inter, union, out=np.zeros_like(inter, dtype=float), where=union > 0)
    acc = np.divide(inter, area_gt, out=np.zeros_like(inter, dtype=float), where=area_gt > 0)

    valid = area_gt > 0
    mIoU = float(iou[valid].mean()) if np.any(valid) else 0.0
    mAcc = float(acc[valid].mean()) if np.any(valid) else 0.0
    aAcc = float(inter.sum() / max(area_gt.sum(), 1))
    freq = area_gt / max(area_gt.sum(), 1)
    fwIoU = float((freq * iou).sum())

    return dict(mIoU=mIoU, mAcc=mAcc, aAcc=aAcc, fwIoU=fwIoU,
                iou=iou, acc=acc, area_gt=area_gt, area_pred=area_pr, area_inter=inter)

def evaluate_dir(model, img_dir, gt_dir, basenames, num_classes, remap_lut=None):
    conf = np.zeros((num_classes, num_classes), dtype=np.int64)
    missing = 0
    for base in tqdm(basenames, ncols=100, desc=os.path.basename(img_dir) or img_dir):
        img_path = os.path.join(img_dir, base + '.jpg')
        gt_path  = os.path.join(gt_dir,  base + '.png')
        if not (os.path.isfile(img_path) and os.path.isfile(gt_path)):
            missing += 1
            continue

        res = inference_model(model, img_path)
        if hasattr(res, 'pred_sem_seg'):
            pred = res.pred_sem_seg.data.squeeze(0).to('cpu').numpy().astype(np.int64)
        elif hasattr(res, 'seg_logits'):
            pred = torch.argmax(res.seg_logits.data, dim=1).squeeze(0).to('cpu').numpy().astype(np.int64)
        else:
            raise AttributeError('No pred_sem_seg or seg_logits in result.')

        gt = load_gt_png(gt_path, remap_lut=remap_lut, num_classes=num_classes, ignore_index=IGNORE_INDEX)
        if pred.shape != gt.shape:
            pred = cv2.resize(pred, (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_NEAREST)

        fast_confusion_update(conf, pred, gt, num_classes, ignore_index=IGNORE_INDEX)

    return metrics_from_confusion(conf), missing

def main():
    os.makedirs(SAVE_DIR, exist_ok=True)
    register_all_modules()
    model = init_model(CONFIG_FILE, CHECKPOINT, device=DEVICE)

    classes = model.dataset_meta.get('classes', None)
    num_classes = len(classes) if classes is not None else model.decode_head.num_classes
    print(f'[INFO] Device: {DEVICE} | num_classes={num_classes}')
    print(f'[INFO] GT dir: {GT_DIR}')
    for k, v in IMAGE_DIRS.items():
        print(f'[INFO] "{k}" -> {v}')
        
    probe_paths = glob.glob(os.path.join(GT_DIR, '*.png'))[:10]
    needs_remap = False
    for p in probe_paths:
        g = load_gt_png(p, remap_lut=None, num_classes=None, ignore_index=IGNORE_INDEX)
        if np.any((g != IGNORE_INDEX) & ((g < 0) | (g >= num_classes))):
            needs_remap = True
            break
        
    remap_lut = None
    if needs_remap:
        remap_lut = build_label_lut(COCO_STUFF_LABEL_MAP, ignore_index=IGNORE_INDEX)
        # sanity: the remapped range should match model classes
        max_after = remap_lut.max()
        if max_after != num_classes - 1:
            print(f'[WARN] Remap LUT max={max_after}, but num_classes-1={num_classes-1}. '
                'Verify your mapping matches the model labels.')
        print('[INFO] Applying COCO_STUFF_LABEL_MAP to GT.')
    else:
        print('[INFO] GT appears already remapped; no label map applied.') 
               
    # -------- Build a common intersection of basenames --------
    gt_bases   = basenames_in(GT_DIR, 'png')
    img_bag = [basenames_in(p, 'jpg') for p in IMAGE_DIRS.values()]
    common = gt_bases.copy()
    for s in img_bag:
        common &= s

    if not common:
        raise RuntimeError('No common basenames across GT and all IMAGE_DIRS. Check folders.')

    common = sorted(common)
    if MAX_SAMPLES is not None:
        common = common[:MAX_SAMPLES]

    print(f'[INFO] Using {len(common)} common images for all dirs (fair comparison).')

    # -------- Evaluate each directory --------
    results = {}
    for name, img_dir in IMAGE_DIRS.items():
        print(f'\n=== Evaluating: {name} ===')
        out, missing = evaluate_dir(model, img_dir, GT_DIR, common, num_classes, remap_lut=remap_lut)
        results[name] = out
        print(f'[{name}] mIoU={out["mIoU"]:.4f}  mAcc={out["mAcc"]:.4f}  aAcc={out["aAcc"]:.4f}  fwIoU={out["fwIoU"]:.4f}')
        if missing > 0:
            print(f'[{name}] WARNING: {missing} missing pairs (should be 0 because of intersection).')

    # -------- Optional: deltas vs "Clean" --------
    if 'Clean' in results:
        base = results['Clean']
        print('\n--- Drops vs Clean (absolute) ---')
        for name, out in results.items():
            diou = out['mIoU'] - base['mIoU']
            dacc = out['aAcc'] - base['aAcc']
            print(f'{name:<14s} ΔmIoU={diou:+.4f}  ΔaAcc={dacc:+.4f}')

    # -------- Save CSVs --------
    if SAVE_SUMMARY_CSV:
        import csv
        sum_path = os.path.join(SAVE_DIR, 'summary.csv')
        base_miou = results.get('Clean', {}).get('mIoU', None)
        base_aacc = results.get('Clean', {}).get('aAcc', None)
        
        with open(sum_path, 'w', newline='') as f:
            w = csv.writer(f)
            w.writerow(['name', 'mIoU', 'mAcc', 'aAcc', 'fwIoU', 'ΔmIoU_vs_Clean', 'ΔaAcc_vs_Clean'])
            for name, out in results.items():
                d_miou = '' if base_miou is None else f'{(out["mIoU"] - base_miou):.6f}'
                d_aacc = '' if base_aacc is None else f'{(out["aAcc"] - base_aacc):.6f}'

                w.writerow([name, f'{out["mIoU"]:.6f}', f'{out["mAcc"]:.6f}', f'{out["aAcc"]:.6f}', f'{out["fwIoU"]:.6f}', d_miou, d_aacc])
        print(f'[INFO] Wrote {sum_path}')

    if SAVE_PER_CLASS_IOU_CSV:
        import csv
        pci_path = os.path.join(SAVE_DIR, 'per_class_iou.csv')
        names = list(results.keys())
        with open(pci_path, 'w', newline='') as f:
            w = csv.writer(f)
            header = ['class_id', 'class_name'] + [f'IoU_{n}' for n in names]
            if 'Clean' in results:
                header += [f'IoU_Delta_vs_Clean_{n}' for n in names if n != 'Clean']
            w.writerow(header)

            for cid in range(num_classes):
                row = [cid, classes[cid] if classes else f'class_{cid}']
                ious = [results[n]['iou'][cid] for n in names]
                row += [f'{x:.6f}' for x in ious]
                if 'Clean' in results:
                    base = results['Clean']['iou'][cid]
                    row += [f'{results[n]["iou"][cid]-base:.6f}' for n in names if n != 'Clean']
                w.writerow(row)
        print(f'[INFO] Wrote {pci_path}')

if __name__ == '__main__':
    main()
